# Worksheet 08

Name:  Houjie Jiang  
UID: U65333668

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[6.992680246264931, 3.363519597945414, 4.734598726945719, 5.496461392885869, 3.338438726152716, 5.767959047469694, 6.078319439944337, 6.092467480924049, 6.078443505238057, 2.9778370407394763]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[6.980374218448478, 9.30532481440615, 8.749665876368374, 9.465630629996092, 5.339999740095845, 8.96475734793362, 8.18195371897909, 6.088293234173545, 8.295240722806557, 7.938565141130528]


# ------------------------------------------------------------------------

b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[2.9778370407394763, 7.938565141130528, 8.295240722806557, 6.088293234173545, 6.078443505238057, 6.092467480924049, 6.078319439944337, 8.18195371897909, 8.96475734793362, 5.767959047469694]


# ------------------------------------------------------------------------

c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

- Number of Components: 2
- Means (μ1, μ2): 5, 8
- Variances (σ^2): 1
- Mixture proportion (p1, p2): 0.5, 0.5

# ------------------------------------------------------------------------

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [4]:
import warnings
warnings. filterwarnings('ignore')

kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2))]
mean = [ sum(s1) / len(s1) , sum(s2) / len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[2.9778370407394763, 6.088293234173545, 6.078443505238057, 6.092467480924049, 6.078319439944337, 5.767959047469694]
[7.938565141130528, 8.295240722806557, 8.18195371897909, 8.96475734793362]
P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 5.513886624748193,  mean_2 = 8.345129232712448
var_1 = 1.2996846538002942,  var_2 = 0.14458711835978738


- P(S_1) = 0.5,  P(S_2) = 0.5
- mean_1 = 5.513886624748193,  mean_2 = 8.345129232712448
- var_1 = 1.2996846538002942,  var_2 = 0.14458711835978738

P and mean are close to the true values. Var_2 is far from the true value.

# ------------------------------------------------------------------------

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [5]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append((pdf_i[0] * prob_s[0]) / prob_x)
    prob_s1_x.append((pdf_i[1] * prob_s[1]) / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  2.9778370407394763
probability of observing that point if it came from cluster 0 =  0.0457387973555883
probability of observing that point if it came from cluster 1 =  1.623552740140713e-299
point =  7.938565141130528
probability of observing that point if it came from cluster 0 =  0.05386507531765763
probability of observing that point if it came from cluster 1 =  0.05294784083534032
point =  8.295240722806557
probability of observing that point if it came from cluster 0 =  0.03108879407134524
probability of observing that point if it came from cluster 1 =  2.599730580525602
point =  6.088293234173545
probability of observing that point if it came from cluster 0 =  0.2783923503512193
probability of observing that point if it came from cluster 1 =  3.435390871745785e-53
point =  6.078443505238057
probability of observing that point if it came from cluster 0 =  0.27931834116737403
probability of observing that point if it came from cluster 1 =  1.1835136155030076e-53
point =  6

### Comment
Based on the estimated probabilities, the assignment would be:
- point 1 belongs to s1
- point 2 belongs to s1
- point 3 belongs to s2
- point 4 belongs to s1
- point 5 belongs to s1
- point 6 belongs to s1
- point 7 belongs to s1
- point 8 belongs to s2
- point 9 belongs to s1
- point 10 belongs to s1

The true assignment is:
- point 1 belongs to s1
- point 2 belongs to s2
- point 3 belongs to s2
- point 4 belongs to s2
- point 5 belongs to s1
- point 6 belongs to s1
- point 7 belongs to s1
- point 8 belongs to s2
- point 9 belongs to s2
- point 10 belongs to s1

So the true assignment and the estimated assignment are mostly same.

# ------------------------------------------------------------------------

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [6]:
prob_s = [sum(prob_s0_x) / len(prob_s0_x), sum(prob_s1_x) / len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), 
        sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

var = [sum([x * ((d - mean[0]) ** 2) for x, d in zip(prob_s0_x, data)]) / sum(prob_s0_x),
       sum([x * ((d - mean[1]) ** 2) for x, d in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.763819012403993,  P(S_2) = 0.23618098759600717
mean_1 = 6.167513915493231,  mean_2 = 8.195068111598895
var_1 = 2.6381002832858322,  var_2 = 0.020410955856582248


### Comment
- Mean values don't change a lot.
- P values and variance values changes a lot towards the wrong direction.

# ------------------------------------------------------------------------

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [7]:
prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append((pdf_i[0] * prob_s[0]) / prob_x)
    prob_s1_x.append((pdf_i[1] * prob_s[1]) / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

point =  2.9778370407394763
probability of observing that point if it came from cluster 0 =  0.07280752168524403
probability of observing that point if it came from cluster 1 =  0.0
point =  7.938565141130528
probability of observing that point if it came from cluster 0 =  0.12071251185326509
probability of observing that point if it came from cluster 1 =  9.942558661991547e-34
point =  8.295240722806557
probability of observing that point if it came from cluster 0 =  0.10923552247838828
probability of observing that point if it came from cluster 1 =  0.00011501607565655261
point =  6.088293234173545
probability of observing that point if it came from cluster 0 =  0.1511551501268281
probability of observing that point if it came from cluster 1 =  0.0
point =  6.078443505238057
probability of observing that point if it came from cluster 0 =  0.15113715023485108
probability of observing that point if it came from cluster 1 =  0.0
point =  6.092467480924049
probability of observing that p

### Comment
- Most probability distribution become 1 and 0, indicating less uncertainty on points assignment.

# ------------------------------------------------------------------------

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

### Comment
Based on the estimated probabilities above, the result would be:
- point 1 belongs to cluster 0
- point 2 belongs to cluster 0
- point 3 belongs to cluster 0
- point 4 belongs to cluster 0
- point 5 belongs to cluster 0
- point 6 belongs to cluster 0
- point 7 belongs to cluster 0
- point 8 belongs to cluster 1
- point 9 belongs to cluster 0
- point 10 belongs to cluster 0